In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

class MultimodalGraphDataset(Dataset):
    def __init__(self, text_data, image_data, audio_data, adjacency_matrices, labels):
        self.text_data = torch.tensor(text_data, dtype=torch.float32)
        self.image_data = torch.tensor(image_data, dtype=torch.float32)
        self.audio_data = torch.tensor(audio_data, dtype=torch.float32)
        self.adjacency_matrices = torch.tensor(adjacency_matrices, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (self.text_data[idx], self.image_data[idx], self.audio_data[idx],
                self.adjacency_matrices[idx], self.labels[idx])


class MGCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MGCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, features, adjacency_matrix):
        degree_matrix = torch.diag_embed(torch.sum(adjacency_matrix, dim=-1))
        normalized_adjacency = torch.bmm(torch.inverse(degree_matrix), adjacency_matrix)
        propagation = torch.bmm(normalized_adjacency, features)
        output = torch.bmm(propagation, self.weight.unsqueeze(0).expand(features.size(0), -1, -1))
        return torch.relu(output)


class MGCNModel(nn.Module):
    def __init__(self, input_dims, hidden_dim, output_dim):
        super(MGCNModel, self).__init__()
        self.text_layer = MGCNLayer(input_dims['text'], hidden_dim)
        self.image_layer = MGCNLayer(input_dims['image'], hidden_dim)
        self.audio_layer = MGCNLayer(input_dims['audio'], hidden_dim)
        self.fc = nn.Linear(3 * hidden_dim, output_dim)

    def forward(self, text_features, image_features, audio_features, adjacency_matrix):
        text_out = self.text_layer(text_features, adjacency_matrix)
        image_out = self.image_layer(image_features, adjacency_matrix)
        audio_out = self.audio_layer(audio_features, adjacency_matrix)
        concatenated = torch.cat((text_out, image_out, audio_out), dim=-1)
        concatenated = concatenated.mean(dim=1)
        return self.fc(concatenated)


def train_mgcn_model(train_loader, model, num_epochs, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for text_data, image_data, audio_data, adjacency_matrix, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(text_data, image_data, audio_data, adjacency_matrix)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")


def main_multimodal():
    text_data = np.random.rand(100, 32, 300)
    image_data = np.random.rand(100, 32, 2048)
    audio_data = np.random.rand(100, 32, 128)
    adjacency_matrices = np.random.rand(100, 32, 32)
    labels = np.random.randint(0, 2, size=100)

    train_size = int(0.8 * len(labels))
    train_dataset = MultimodalGraphDataset(
        text_data[:train_size], image_data[:train_size], audio_data[:train_size],
        adjacency_matrices[:train_size], labels[:train_size])
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

    model = MGCNModel(
        input_dims={'text': 300, 'image': 2048, 'audio': 128},
        hidden_dim=64, output_dim=2)

    train_mgcn_model(train_loader, model, num_epochs=200, learning_rate=0.01)

if __name__ == "__main__":
    main_multimodal()


Epoch 1/200, Loss: 0.8514
Epoch 2/200, Loss: 0.7285
Epoch 3/200, Loss: 0.6954
Epoch 4/200, Loss: 0.7131
Epoch 5/200, Loss: 0.6780
Epoch 6/200, Loss: 0.6599
Epoch 7/200, Loss: 0.8207
Epoch 8/200, Loss: 0.7330
Epoch 9/200, Loss: 0.7394
Epoch 10/200, Loss: 0.6773
Epoch 11/200, Loss: 0.6601
Epoch 12/200, Loss: 0.6744
Epoch 13/200, Loss: 0.6754
Epoch 14/200, Loss: 0.6575
Epoch 15/200, Loss: 0.7604
Epoch 16/200, Loss: 0.6974
Epoch 17/200, Loss: 0.6967
Epoch 18/200, Loss: 0.6973
Epoch 19/200, Loss: 0.6968
Epoch 20/200, Loss: 0.6973
Epoch 21/200, Loss: 0.6971
Epoch 22/200, Loss: 0.6967
Epoch 23/200, Loss: 0.6226
Epoch 24/200, Loss: 0.7622
Epoch 25/200, Loss: 0.6752
Epoch 26/200, Loss: 0.6979
Epoch 27/200, Loss: 0.7232
Epoch 28/200, Loss: 0.7222
Epoch 29/200, Loss: 0.6752
Epoch 30/200, Loss: 0.6977
Epoch 31/200, Loss: 0.7237
Epoch 32/200, Loss: 0.7237
Epoch 33/200, Loss: 0.6502
Epoch 34/200, Loss: 0.7245
Epoch 35/200, Loss: 0.6738
Epoch 36/200, Loss: 0.6977
Epoch 37/200, Loss: 0.6744
Epoch 38/2

In [7]:
import torch
import torch.nn as nn
import numpy as np
import networkx as nx
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

class MGCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MGCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.bias = nn.Parameter(torch.FloatTensor(output_dim))
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, node_features, adj_matrix):
        # Graph convolution: H = D^(-1/2) * A * D^(-1/2) * H * W + b
        degree_matrix = torch.diag(torch.sum(adj_matrix, dim=1))
        degree_matrix_inv_sqrt = torch.linalg.inv(torch.sqrt(degree_matrix))
        normalized_adj = torch.mm(degree_matrix_inv_sqrt, torch.mm(adj_matrix, degree_matrix_inv_sqrt))
        convolved_features = torch.mm(normalized_adj, node_features)
        return torch.mm(convolved_features, self.weight) + self.bias


class MultimodalMGCN(nn.Module):
    def __init__(self, input_dims, hidden_dim, output_dim, modalities):
        super(MultimodalMGCN, self).__init__()
        self.modalities = modalities
        self.input_layers = nn.ModuleDict({
            modality: MGCNLayer(input_dims[modality], hidden_dim) for modality in modalities
        })
        self.shared_layer = MGCNLayer(hidden_dim * len(modalities), output_dim)

    def forward(self, multimodal_features, multimodal_adjs):
        modality_features = []
        for modality in self.modalities:
            features = multimodal_features[modality]
            adj = multimodal_adjs[modality]
            updated_features = self.input_layers[modality](features, adj)
            modality_features.append(updated_features)
        
        concatenated_features = torch.cat(modality_features, dim=1)
        final_features = self.shared_layer(concatenated_features, sum(multimodal_adjs.values()))
        return final_features


# Dataset Preparation for Multimodal Data
def prepare_multimodal_data(dataset, modalities):
    multimodal_features = {modality: None for modality in modalities}
    multimodal_adjs = {modality: None for modality in modalities}
    
    for modality in modalities:
        feature_file = f'Datasets/{dataset}/{modality}_features.csv'
        adj_file = f'Datasets/{dataset}/{modality}_adj.csv'
        
        features = pd.read_csv(feature_file, header=None).values
        adj = pd.read_csv(adj_file, header=None).values
        
        multimodal_features[modality] = torch.tensor(features, dtype=torch.float32)
        multimodal_adjs[modality] = torch.tensor(adj, dtype=torch.float32)
    
    return multimodal_features, multimodal_adjs


# Main Function
def main_mgcn(dataset, modalities, input_dims, hidden_dim, output_dim):
    multimodal_features, multimodal_adjs = prepare_multimodal_data(dataset, modalities)
    
    model = MultimodalMGCN(input_dims, hidden_dim, output_dim, modalities)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Example training loop (dummy data for demonstration)
    for epoch in range(100):  # Replace with actual data loader
        optimizer.zero_grad()
        
        outputs = model(multimodal_features, multimodal_adjs)
        labels = torch.randint(0, 2, (outputs.shape[0],))  # Dummy labels
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Dataset Preparation for Multimodal Data with Dummy Data
def prepare_multimodal_data_dummy(modalities, input_dims, num_nodes):
    multimodal_features = {modality: None for modality in modalities}
    multimodal_adjs = {modality: None for modality in modalities}
    
    for modality in modalities:
        # Generate random feature matrices
        features = np.random.rand(num_nodes, input_dims[modality])
        # Generate random adjacency matrices
        adj = np.random.randint(0, 2, (num_nodes, num_nodes))
        # Ensure symmetry for adjacency matrices
        adj = (adj + adj.T) / 2
        adj[adj > 1] = 1  # Ensure it's binary
        
        multimodal_features[modality] = torch.tensor(features, dtype=torch.float32)
        multimodal_adjs[modality] = torch.tensor(adj, dtype=torch.float32)
    
    return multimodal_features, multimodal_adjs


def main_mgcn_dummy(modalities, input_dims, hidden_dim, output_dim, num_nodes):
    multimodal_features, multimodal_adjs = prepare_multimodal_data_dummy(modalities, input_dims, num_nodes)
    
    model = MultimodalMGCN(input_dims, hidden_dim, output_dim, modalities)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Example training loop with dummy data
    for epoch in range(200):  # Shortened for demonstration
        optimizer.zero_grad()
        
        # Get the model outputs
        outputs = model(multimodal_features, multimodal_adjs)
        
        # Generate dummy labels for binary classification (0 or 1)
        labels = torch.randint(0, 2, (outputs.shape[0],))  # Dummy binary labels
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Get predicted class labels (argmax for binary classification)
        _, predicted = torch.max(outputs, dim=1)
        
        # Calculate accuracy, precision, recall, and F1 score
        accuracy = accuracy_score(labels.numpy(), predicted.numpy())
        precision = precision_score(labels.numpy(), predicted.numpy())
        recall = recall_score(labels.numpy(), predicted.numpy())
        f1 = f1_score(labels.numpy(), predicted.numpy())
        
        # Backpropagate and optimize the model
        loss.backward()
        optimizer.step()
        
        # Print the metrics for this epoch
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}, "
              f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


# Run with dummy data
if __name__ == '__main__':
    modalities = ['text', 'speech', 'image']
    input_dims = {'text': 300, 'speech': 128, 'image': 512}  # Example input dimensions for each modality
    hidden_dim = 64
    output_dim = 2  # Binary classification
    num_nodes = 100  # Example number of nodes in the graph
    main_mgcn_dummy(modalities, input_dims, hidden_dim, output_dim, num_nodes)



# if __name__ == '__main__':
#     dataset = "MultimodalDataset"
#     modalities = ['text', 'speech', 'image']
#     input_dims = {'text': 300, 'speech': 128, 'image': 512}  # Example input dimensions for each modality
#     hidden_dim = 64
#     output_dim = 2  # Binary classification
#     main_mgcn(dataset, modalities, input_dims, hidden_dim, output_dim)


Epoch 1, Loss: 0.7267, Accuracy: 0.5700, Precision: 0.5700, Recall: 1.0000, F1 Score: 0.7261
Epoch 2, Loss: 1.5403, Accuracy: 0.4500, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 3, Loss: 0.6903, Accuracy: 0.6000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 4, Loss: 0.8824, Accuracy: 0.4400, Precision: 0.4400, Recall: 1.0000, F1 Score: 0.6111
Epoch 5, Loss: 0.8355, Accuracy: 0.5400, Precision: 0.5400, Recall: 1.0000, F1 Score: 0.7013
Epoch 6, Loss: 0.7497, Accuracy: 0.5200, Precision: 0.5200, Recall: 1.0000, F1 Score: 0.6842
Epoch 7, Loss: 0.6980, Accuracy: 0.4700, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 8, Loss: 0.7561, Accuracy: 0.5000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 9, Loss: 0.7655, Accuracy: 0.5100, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 10, Loss: 0.7277, Accuracy: 0.4800, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 11, Loss: 0.6988, Accuracy: 0.4800, Precision: 0.4800, Recall: 

C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

Epoch 21, Loss: 0.7021, Accuracy: 0.5300, Precision: 0.5300, Recall: 1.0000, F1 Score: 0.6928
Epoch 22, Loss: 0.7450, Accuracy: 0.5100, Precision: 0.5100, Recall: 1.0000, F1 Score: 0.6755
Epoch 23, Loss: 0.7264, Accuracy: 0.5200, Precision: 0.5200, Recall: 1.0000, F1 Score: 0.6842
Epoch 24, Loss: 0.7103, Accuracy: 0.4700, Precision: 0.4700, Recall: 1.0000, F1 Score: 0.6395
Epoch 25, Loss: 0.6908, Accuracy: 0.5400, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 26, Loss: 0.7621, Accuracy: 0.4800, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 27, Loss: 0.7589, Accuracy: 0.4800, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 28, Loss: 0.7253, Accuracy: 0.4200, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 29, Loss: 0.6986, Accuracy: 0.5300, Precision: 0.5300, Recall: 1.0000, F1 Score: 0.6928
Epoch 30, Loss: 0.8186, Accuracy: 0.4500, Precision: 0.4500, Recall: 1.0000, F1 Score: 0.6207
Epoch 31, Loss: 0.7584, Accuracy: 0.5000, Precision: 0.5000,

C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

Epoch 40, Loss: 0.6831, Accuracy: 0.5700, Precision: 0.5700, Recall: 1.0000, F1 Score: 0.7261
Epoch 41, Loss: 0.7051, Accuracy: 0.4400, Precision: 0.4400, Recall: 1.0000, F1 Score: 0.6111
Epoch 42, Loss: 0.6991, Accuracy: 0.4700, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 43, Loss: 0.7095, Accuracy: 0.4700, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 44, Loss: 0.6953, Accuracy: 0.5100, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 45, Loss: 0.6928, Accuracy: 0.5200, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 46, Loss: 0.7018, Accuracy: 0.4000, Precision: 0.4000, Recall: 1.0000, F1 Score: 0.5714
Epoch 47, Loss: 0.6947, Accuracy: 0.4900, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 48, Loss: 0.7061, Accuracy: 0.4400, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 49, Loss: 0.6952, Accuracy: 0.4000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 50, Loss: 0.6765, Accuracy: 0.5900, Precision: 0.5900,

...

C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

Epoch 163, Loss: 0.6956, Accuracy: 0.4800, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 164, Loss: 0.6865, Accuracy: 0.5600, Precision: 0.5600, Recall: 1.0000, F1 Score: 0.7179
Epoch 165, Loss: 0.7193, Accuracy: 0.5400, Precision: 0.5400, Recall: 1.0000, F1 Score: 0.7013
Epoch 166, Loss: 0.7252, Accuracy: 0.5300, Precision: 0.5300, Recall: 1.0000, F1 Score: 0.6928
Epoch 167, Loss: 0.7080, Accuracy: 0.5000, Precision: 0.5000, Recall: 1.0000, F1 Score: 0.6667
Epoch 168, Loss: 0.6903, Accuracy: 0.5400, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 169, Loss: 0.7344, Accuracy: 0.5000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 170, Loss: 0.7552, Accuracy: 0.4800, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 171, Loss: 0.7055, Accuracy: 0.4900, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 172, Loss: 0.6907, Accuracy: 0.5400, Precision: 0.5400, Recall: 1.0000, F1 Score: 0.7013
Epoch 173, Loss: 0.7410, Accuracy: 0.5100, Precisi

C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

Epoch 184, Loss: 0.7093, Accuracy: 0.4100, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 185, Loss: 0.6908, Accuracy: 0.5400, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 186, Loss: 0.7021, Accuracy: 0.5000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 187, Loss: 0.6989, Accuracy: 0.4800, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 188, Loss: 0.6933, Accuracy: 0.5100, Precision: 0.5100, Recall: 1.0000, F1 Score: 0.6755
Epoch 189, Loss: 0.6977, Accuracy: 0.5200, Precision: 0.5200, Recall: 1.0000, F1 Score: 0.6842
Epoch 190, Loss: 0.7176, Accuracy: 0.4700, Precision: 0.4700, Recall: 1.0000, F1 Score: 0.6395
Epoch 191, Loss: 0.7071, Accuracy: 0.3900, Precision: 0.3900, Recall: 1.0000, F1 Score: 0.5612
Epoch 192, Loss: 0.7189, Accuracy: 0.4900, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 193, Loss: 0.7595, Accuracy: 0.4700, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 194, Loss: 0.7281, Accuracy: 0.4600, Precisi

C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
